In [16]:
# NAIVE / POTENTIALLY BIASED (example)
def score_applicant_naive(applicant: dict) -> float:
    """
    Naive scoring that (dangerously) uses gender as a direct multiplier.
    applicant keys: education_years, years_experience, skill_score (0-100),
                    certifications_count, gender ('male'|'female'|'other')
    """
    base = 0.4 * applicant['education_years'] + \
           0.4 * applicant['years_experience'] + \
           0.002 * applicant['skill_score'] + \
           0.5 * applicant['certifications_count']
    # RISK: injecting gender factor directly can cause discrimination
    gender = applicant.get('gender', '').lower()
    if gender == 'male':
        base *= 1.05   # unfair boost for males
    elif gender == 'female':
        base *= 0.95   # unfair penalty for females
    # return a numeric score
    return round(base, 3)

In [17]:
# SAFER: scoring without gender and with normalization
from typing import Dict
import math

def score_applicant_safe(applicant: Dict[str, float]) -> float:
    """
    Safer applicant scoring that does NOT use gender or other protected attributes.
    Expected applicant keys: education_years, years_experience, skill_score (0-100),
    certifications_count. Missing keys default to 0.

    Returns:
        float: normalized score in range [0, 100]
    """
    # extract features with safe defaults
    edu = float(applicant.get('education_years', 0))
    exp = float(applicant.get('years_experience', 0))
    skill = float(applicant.get('skill_score', 0))  # 0-100
    certs = float(applicant.get('certifications_count', 0))

    # feature scaling (simple): map features to roughly comparable ranges
    # education_years: assume 0-20 -> normalized to 0-1
    edu_norm = min(edu / 20.0, 1.0)
    # experience: 0-30 years -> 0-1
    exp_norm = min(exp / 30.0, 1.0)
    # skill: already 0-100 -> 0-1
    skill_norm = min(max(skill, 0.0), 100.0) / 100.0
    # certifications: treat 0-10 -> 0-1
    cert_norm = min(certs / 10.0, 1.0)

    # weighted sum (weights sum to 1)
    w_edu, w_exp, w_skill, w_cert = 0.2, 0.3, 0.4, 0.1
    raw = w_edu * edu_norm + w_exp * exp_norm + w_skill * skill_norm + w_cert * cert_norm

    # map to 0-100 score
    score = raw * 100.0
    # optional: apply a small smoothing
    return round(score, 2)

In [18]:
# bias_checks.py
from collections import defaultdict
from copy import deepcopy
from typing import List, Dict, Callable

def group_stats(applicants: List[Dict], score_fn: Callable, threshold: float = 50.0, group_key: str = 'gender'):
    """
    Compute average score and acceptance rate per group.
    threshold: score threshold to 'accept' applicant.
    """
    groups = defaultdict(list)
    for a in applicants:
        g = a.get(group_key, 'unknown')
        groups[g].append(a)

    results = {}
    for g, items in groups.items():
        scores = [score_fn(x) for x in items]
        avg_score = sum(scores) / len(scores) if scores else 0.0
        accept_rate = sum(1 for s in scores if s >= threshold) / len(scores) if scores else 0.0
        results[g] = {'avg_score': avg_score, 'accept_rate': accept_rate, 'n': len(scores)}
    return results

def counterfactual_invariance_check(applicants: List[Dict], score_fn: Callable, sensitive_attr: str = 'gender', allowed_diff: float = 1e-6):
    """
    For each applicant, flip the sensitive attribute and recompute score.
    Count how many change beyond allowed_diff.
    Returns fraction changed.
    Note: This test is for deterministic score functions that read the sensitive attribute.
    """
    changed = 0
    total = 0
    for a in applicants:
        total += 1
        a_copy = deepcopy(a)
        # flip gender placeholder (male<->female), leave 'other' unchanged for simplicity
        if a_copy.get(sensitive_attr) == 'male':
            a_copy[sensitive_attr] = 'female'
        elif a_copy.get(sensitive_attr) == 'female':
            a_copy[sensitive_attr] = 'male'
        else:
            # try swapping to male
            a_copy[sensitive_attr] = 'male'
        s_orig = score_fn(a)
        s_cf = score_fn(a_copy)
        if abs(s_orig - s_cf) > allowed_diff:
            changed += 1
    return {'changed': changed, 'total': total, 'fraction_changed': changed/total if total else 0.0}

In [19]:
# simulate_bias.py
import random
# Note: group_stats and counterfactual_invariance_check are defined in another notebook cell (bias_checks.py content).
# Similarly, score_applicant_naive and score_applicant_safe are defined in earlier cells.
# In a Jupyter notebook you can use those functions directly without importing a module.

def make_synthetic(n=500, bias_in_labels=False):
    names = ['A','B','C','D']
    genders = ['male', 'female']
    data = []
    for i in range(n):
        gender = random.choice(genders)
        edu = random.randint(10, 20)
        exp = random.randint(0, 20)
        skill = random.uniform(30, 95)
        certs = random.randint(0, 5)
        # optionally bias labels: e.g., lower skill for female (simulating historical bias)
        if bias_in_labels and gender == 'female':
            skill -= 5.0  # simulating biased historical outcomes
        data.append({'education_years': edu, 'years_experience': exp, 'skill_score': skill, 'certifications_count': certs, 'gender': gender})
    return data

# create data
applicants = make_synthetic(1000, bias_in_labels=True)

# compute stats
print("Naive scorer stats:")
print(group_stats(applicants, score_applicant_naive, threshold=55.0, group_key='gender'))
print("Naive counterfactual changes:", counterfactual_invariance_check(applicants, score_applicant_naive))

print("\nSafe scorer stats:")
print(group_stats(applicants, score_applicant_safe, threshold=55.0, group_key='gender'))
print("Safe counterfactual changes:", counterfactual_invariance_check(applicants, score_applicant_safe))

Naive scorer stats:
{'male': {'avg_score': 11.704725806451613, 'accept_rate': 0.0, 'n': 496}, 'female': {'avg_score': 10.814525793650793, 'accept_rate': 0.0, 'n': 504}}
Naive counterfactual changes: {'changed': 1000, 'total': 1000, 'fraction_changed': 1.0}

Safe scorer stats:
{'male': {'avg_score': 51.92, 'accept_rate': 0.4032258064516129, 'n': 496}, 'female': {'avg_score': 50.40561507936508, 'accept_rate': 0.31547619047619047, 'n': 504}}
Safe counterfactual changes: {'changed': 0, 'total': 1000, 'fraction_changed': 0.0}


In [20]:
def test_counterfactual_safe():
    base = {'education_years': 16, 'years_experience': 5, 'skill_score': 80.0, 'certifications_count': 2, 'gender': 'female'}
    copy = base.copy()
    copy['gender'] = 'male'
    assert score_applicant_safe(base) == score_applicant_safe(copy)